In [1]:
!pip install tensorflow-datasets

In [2]:
!python -m pip install --upgrade pip

Requirement already up-to-date: pip in c:\users\affinity\anaconda3\lib\site-packages (19.3.1)


In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.models import load_model
from datetime import datetime
#tf.enable_eager_execution()

Using TensorFlow backend.


In [4]:
train_data = tfds.load(name='cifar10:3.*.*', split='train[:80%]', as_supervised=True)
valid_data = tfds.load(name='cifar10:3.*.*', split='train[80%:]', as_supervised=True)
test_data = tfds.load(name='cifar10:3.*.*', split='test', as_supervised=True)

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same', input_shape=(32, 32, 3)),
    tf.keras.layers.BatchNormalization(),  # Gradient Vanishing / Gradient Exploding 이 일어나지 않도록 하는 아이디어 중의 하나이다
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='softmax')
    
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 128)       3584      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 128)       512       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 256)       295168    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 256)       590080    
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 256)         0

In [6]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

train_data = train_data.map(scale).shuffle(50000).repeat(4).batch(64) 
valid_data = valid_data.map(scale).batch(64)
test_data = test_data.map(scale).batch(64)

In [8]:
dt = datetime.now().strftime('%Y%m%d-%H%M%S')

cp_callback = tf.keras.callbacks.ModelCheckpoint(   
    'checkpoints/{}.hdf5'.format(dt),
    monitor='val_accuracy',
    save_best_only=True)

In [9]:
class MyCallback(tf.keras.callbacks.Callback):

    def on_epoch_start(self, epoch, logs=None):
        print('Epoch {} begins at {}: {}'.format(epoch, datetime.now().time(), logs))

    def on_epoch_end(self, epoch, logs=None):
        print('Epoch {} ends at {}: {}'.format(epoch, datetime.now().time(), logs))

In [10]:
history = model.fit(train_data,
                    epochs=30,
                    validation_data=valid_data,
                    verbose=0,
                    callbacks=[cp_callback, MyCallback()])

Epoch 0 ends at 10:24:32.239490: {'loss': 1.8896847425937653, 'accuracy': 0.5135125, 'val_loss': 2.413166175602348, 'val_accuracy': 0.5371}
Epoch 1 ends at 10:28:24.251229: {'loss': 0.8889042232871056, 'accuracy': 0.7130625, 'val_loss': 0.9406637021690417, 'val_accuracy': 0.7153}
Epoch 2 ends at 10:32:24.983161: {'loss': 0.5067302008271217, 'accuracy': 0.82461876, 'val_loss': 0.8104464056765198, 'val_accuracy': 0.7601}
Epoch 3 ends at 10:36:31.241278: {'loss': 0.3073736889064312, 'accuracy': 0.8910375, 'val_loss': 0.7781527868122052, 'val_accuracy': 0.7807}
Epoch 4 ends at 10:40:41.474749: {'loss': 0.20795787562802434, 'accuracy': 0.9272938, 'val_loss': 0.8401508302825271, 'val_accuracy': 0.7761}
Epoch 5 ends at 10:44:57.833445: {'loss': 0.1493697081591934, 'accuracy': 0.94720626, 'val_loss': 0.8536502274737996, 'val_accuracy': 0.7926}
Epoch 6 ends at 10:49:16.221621: {'loss': 0.12629481075666846, 'accuracy': 0.956125, 'val_loss': 0.9171093778245768, 'val_accuracy': 0.7914}
Epoch 7 end

In [11]:
model.evaluate(test_data)

157/157 [==============================]1.1560 - accuracy: 0.812 - 0s 83ms/step - loss: 1.1710 - accuracy: 0.812 - 0s 68ms/step - loss: 1.0797 - accuracy: 0.83 - 0s 61ms/step - loss: 1.0152 - accuracy: 0.83 - 0s 57ms/step - loss: 1.0417 - accuracy: 0.84 - 0s 54ms/step - loss: 0.9960 - accuracy: 0.83 - 0s 52ms/step - loss: 1.0145 - accuracy: 0.84 - 0s 51ms/step - loss: 1.1137 - accuracy: 0.82 - 0s 50ms/step - loss: 1.0983 - accuracy: 0.82 - 0s 49ms/step - loss: 1.1317 - accuracy: 0.82 - 1s 48ms/step - loss: 1.0679 - accuracy: 0.83 - 1s 48ms/step - loss: 1.0607 - accuracy: 0.83 - 1s 47ms/step - loss: 1.0683 - accuracy: 0.82 - 1s 46ms/step - loss: 1.0429 - accuracy: 0.83 - 1s 46ms/step - loss: 1.0296 - accuracy: 0.83 - 1s 46ms/step - loss: 1.1040 - accuracy: 0.83 - 1s 46ms/step - loss: 1.1124 - accuracy: 0.83 - 1s 45ms/step - loss: 1.1546 - accuracy: 0.82 - 1s 45ms/step - loss: 1.1522 - accuracy: 0.82 - 1s 45ms/step - loss: 1.1673 - accuracy: 0.82 - 1s 45ms/step - loss: 1.1513 - accuracy:

[1.2631675528872544, 0.814]

In [12]:
m = tf.keras.models.load_model('checkpoints/{}.hdf5'.format(dt))